In [ ]:

# -*- coding: utf-8 -*-
"""
Created on Sun Aug 14 23:25:36 2022

@author: HP
"""

import random
import math
import numpy as np
import pyinputplus as pyip
from deap import base
from deap import creator
from deap import tools

# This function evaluate the weighted distance of customers to DCs
def evaldist(individual):
    # Assign customers to DCs randomly
    c2DC = np.zeros((customN,DCN), dtype=np.uint8)
    c2DC[np.arange(customN) , np.random.randint(0,DCN, size=customN)] = 1
    # Calculate the weighted distance
    for j in range(DCN):
        Dist[j] = 0
        for i in range(customN):
            if c2DC[i,j] == 1:
                Dist[j] += math.sqrt((individual[j*2]-df[i,0])**2+(individual[j*2+1]-df[i,1])**2)*df[i,2]
    return sum(Dist),

customN = int(input("Enter the number of customers?"))
DCN = int(input("Enter the number of distribution centers?"))
indLength = DCN*2
df = np.empty(shape=(customN , 3))
Dist = np.empty(shape=(DCN))
for i in range(customN):
     df[i,0]= pyip.inputInt(prompt= f"Enter the X coordinate of customer {i+1}")
     df[i,1]= pyip.inputInt(prompt= f"Enter the Y coordinate of customer {i+1}")
     df[i,2]= pyip.inputInt(prompt= f"Enter the demand of customer {i+1}")
minvalue = df[:, 0:2].min()
maxvalue = df[:, 0:2].max()
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_xy", random.randint, minvalue, maxvalue)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_xy, indLength)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaldist)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)
pop = toolbox.population(n=100)
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit
    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
CXPB, MUTPB = 0.5, 0.2
    # Extracting all the fitnesses of 
fits = [ind.fitness.values[0] for ind in pop]
g = 0

    # Begin the evolution
while g < 100:
    # A new generation
    g = g + 1
    print("-- Generation %i --" % g)
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    pop[:] = offspring
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    
print(f"Final Objective: {min(fits)}")
print(f"The best individual: {pop[fits.index(min(fits))]}")